In [1]:
import tensorflow as tf

In [35]:
tf.reset_default_graph()
v1 = tf.placeholder(tf.float32, name="v1") 
v2 = tf.placeholder(tf.float32, name="v2")
v3 = tf.add(v1, v2)
vx = tf.Variable(10.0, name="vx")
v4 = tf.add(v3, vx, name="v4_op")
v5=tf.pow(v4,2.5,name='v5')
saver = tf.train.Saver()

#tf.reset_default_graph()
with tf.Session() as sess: 
    sess.run(tf.initialize_all_variables())
    sess.run(vx.assign(tf.add(vx, vx)))
    result = sess.run(v4, feed_dict={v1:12.0, v2:3.3})
    print(result)
    #tf.saved_model.simple_save(sess,'./model/model_ex1',inputs={'in1':v1,'in2':v2},outputs={'out1':v4})
    saver.save(sess, "./model/model_ex3")

35.3


In [33]:
tf.reset_default_graph()
with tf.Session() as sess: 
    saver = tf.train.import_meta_graph("./model/model_ex3.meta")
    saver.restore(sess,tf.train.latest_checkpoint('./model/'))
    #tf.saved_model.load(sess,tags=[tf.saved_model.tag_constants.SERVING],export_dir='./model/model_ex1')
   
  
    for op in tf.get_default_graph().get_operations():
        print(op.name) 
#     print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
    graph=tf.get_default_graph()
    op=graph.get_tensor_by_name('v4_op:0')
    new_op=tf.divide(op,10)
    new_op=tf.identity(new_op,name='new_v5')
    result = sess.run(new_op, 
                      feed_dict={"v1:0": 12.0, "v2:0": 3.3})  
    
    #print(sess.graph_def)
    # the problem with Basenji is that the input feed_dict is not known with pre-trained model
    #print(sess.graph_def)
    new_graph_def=tf.graph_util.extract_sub_graph(sess.graph_def,['v1','v2','vx','v4_op','new_v5'])  # extract subgraph accessible to destination nodes defined here
    
    # this can still output modified graph, but only for visualization
    writer = tf.summary.FileWriter("./model/model_ex3/model_graph", graph=new_graph_def,filename_suffix='new')
    #writer = tf.summary.FileWriter("./model/model_ex3/model_graph", graph=sess.graph,filename_suffix='old')
    print()
    print(result)
    print()
    for _ in new_graph_def.node:
        print(_.name)
   
    

W0905 16:16:44.739183 11996 writer.py:199] Passing a `GraphDef` to the SummaryWriter is deprecated. Pass a `Graph` object instead, such as `sess.graph`.


v1
v2
Add
vx/initial_value
vx
vx/Assign
vx/read
v4_op
v5/y
v5
save/filename/input
save/filename
save/Const
save/SaveV2/tensor_names
save/SaveV2/shape_and_slices
save/SaveV2
save/control_dependency
save/RestoreV2/tensor_names
save/RestoreV2/shape_and_slices
save/RestoreV2
save/Assign
save/restore_all
init
Add_1
Assign

3.53

v1
v2
Add
vx
vx/read
v4_op
truediv/y
truediv
new_v5


In [25]:
graph=tf.get_default_graph()
print()
print(graph.get_all_collection_keys())
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
#graph.get_operations()
graph.get_tensor_by_name('vx:0')

graph.get_operation_by_name('v5')


['trainable_variables', 'variables']
[<tf.Variable 'vx:0' shape=() dtype=float32_ref>]


<tf.Operation 'v5' type=Pow>